# Solving time-dependent Ising Model using Pulser

In [ ]:
import cmath
import numpy as np
import networkx as nx
from scipy.constants import hbar
from collections import Counter
from typing import Union, Literal
from networkx.drawing.nx_agraph import graphviz_layout
from qat.core import Variable, Schedule, Observable, Term, Result
from qat.core.qpu import QPUHandler
from qat.core.variables import ArithExpression, cos, sin, heaviside


## Defining Pasqal's QPU

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser_simulation import Simulation
from pulser.waveforms import CustomWaveform, Waveform
from pulser.devices import Device, VirtualDevice, interaction_coefficients
from pulser.devices import MockDevice
from pulser.devices import Chadoq2
from pulserAQPU import FresnelAQPU, PasqalAQPU
from pprint import pprint

In [ ]:
# Create a device having a rydberg global channel
device = MockDevice
# Create a square 2x2 register with side 4um 
register = Register.square(2, 4, None)
qpu = FresnelAQPU(device, register)

In [ ]:
# You can see the register used by the qpu
qpu.register.draw()

In [ ]:
# Check qpu properties
print(qpu.distances)  # symetric matrix, for each qubit two qubits are at 4um distance and one at 4 * sqrt(2) um 
print(qpu.interactions)  # symetric matrix, C_6/R^6

## Defining an Ising Hamiltonian on MyQLM

On Pulser we can solve problems of shape 
$$ H = \hbar\sum_i \frac{\Omega(t)}{2}(\cos(\phi) \sigma_i^x - \sin(\phi) \sigma_i^y) - \delta(t)n_i + \frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$$
with $\sigma_i^x$ the Pauli $X$ operator applied on qubit $i$ and $n_i = \frac{1-\sigma_i^z}{2}$ with $\sigma_i^z$ the Pauli $Z$ operator applied on qubit $i$

Let's start by defining the parameters and the Hamiltonian

In [ ]:
nqubits = 4
t_variable = Variable("t")  # in ns  
u_variable = Variable("u") # parameter
omega_t = t_variable  # in rad/us
delta_t = 1 - t_variable + u_variable # in rad/us

In [ ]:
# Hamiltonian of a pulse of constant amplitude, zero detuning and phase
t1 = 20  # in ns
H1 = qpu.ising_hamiltonian(nqubits, 1, 0, 0)
print(H1)  # Apply a X gate on each qubit, and ZZ gate whose coefficient is defined from interactions                                         

In [ ]:
# Same hamiltonian with a pi/2 phase
t2 = 20  # in ns
H2 = qpu.ising_hamiltonian(nqubits, 1, 0, np.pi/2)
print(H2)  # Apply a Y gate on each qubit, and ZZ gate whose coefficient is defined from interactions   

In [ ]:
# Hamiltonian with a varying amplitude and detuning
t0 = 10  # in ns
H0 = qpu.ising_hamiltonian(nqubits, omega_t, delta_t, 0)
print(H0)  # Apply a Y gate on each qubit, and ZZ gate whose coefficient is defined from interactions   

The Hamiltonian is implemented as a `Schedule` object. You also have to define the duration of the evolution

In [ ]:
# The drive coefficient should only be 1 or product of heaviside function
# Otherwise the interaction terms will no longer correspond with the register
schedule0 = Schedule(drive=[(1, H0)],
                    tmax=t0)
schedule1 = Schedule(drive=[(1, H1)],
                    tmax=t1)
schedule2 = Schedule(drive=[(1, H2)],
                    tmax=t2)
print(schedule1)

In [ ]:
# the schedules can be stacked to create a pulse sequence
schedule = schedule0 | schedule1 | schedule2
print(schedule)

In [ ]:
# This schedule could also have been defined as below
# Because Schedule sums the input hamiltonian defined in drive
schedule_sum = Schedule(
    drive=[
        (heaviside(t_variable, 0, t0), H0),
        (heaviside(t_variable, t0, t1+t0), H1),
        (heaviside(t_variable, t1+t0, t1+t0+t2), H2)
        ],
    tmax = t1+t0+t2
    )
print(schedule_sum)

From a schedule object we can build jobs to be submit to the QPU. An initial state can be defined in the job, as well as an Hamiltonian to be measured at the output.

In [ ]:
# To sample the final state in the computational basis
job0 = schedule.to_job()
job0_sum = schedule_sum.to_job()

# To evaluate some observable at the end of the computation
H_target = Observable(nqubits, pauli_terms=[Term(1, "XX", [0, 1])])
jobobs = schedule.to_job(observable=H_target)

# Starting from |++++> state
job1 = schedule.to_job(psi_0='++++')

# Starting from a random initial state (simulator only)
vec = np.random.random(2**nqubits)
vec /= np.linalg.norm(vec)
job2 = schedule.to_job(psi_0=vec)

Run the jobs on the qpu

In [ ]:
myqlmResult = qpu.submit_job(job0(u=0), toprint=True, todraw=True, asdict=False)

In [ ]:
myqlmResult.raw_data

In [ ]:
# Count result:
dictResult = qpu.submit_job(job0(u=0), toprint=True, todraw=True, asdict=True)

In [ ]:
dictResult